In [1]:
!pip install -q segmentation-models-pytorch albumentations opencv-python tqdm



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 2.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile
import os

TRAIN_ZIP = "/content/drive/MyDrive/Offroad_Segmentation_Training_Dataset.zip"
TEST_ZIP  = "/content/drive/MyDrive/Offroad_Segmentation_testImages.zip"

EXTRACT_PATH = "/content/dataset"

os.makedirs(EXTRACT_PATH, exist_ok=True)

with zipfile.ZipFile(TRAIN_ZIP, 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_PATH)

with zipfile.ZipFile(TEST_ZIP, 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_PATH)

print("Extraction completed.")


Extraction completed.


In [4]:
!ls /content/dataset


Offroad_Segmentation_testImages  Offroad_Segmentation_Training_Dataset


In [5]:
TRAIN_IMG_DIR = "/content/dataset/Offroad_Segmentation_Training_Dataset/train/Color_Images"
TRAIN_MASK_DIR = "/content/dataset/Offroad_Segmentation_Training_Dataset/train/Segmentation"

VAL_IMG_DIR = "/content/dataset/Offroad_Segmentation_Training_Dataset/val/Color_Images"
VAL_MASK_DIR = "/content/dataset/Offroad_Segmentation_Training_Dataset/val/Segmentation"

TEST_IMG_DIR = "/content/dataset/Offroad_Segmentation_testImages/Color_Images"

In [6]:
import os
import cv2
import numpy as np
import torch
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)


Device: cuda


In [7]:
def get_unique_colors(mask_dir, sample_limit=100):
    colors = set()
    files = os.listdir(mask_dir)[:sample_limit]

    for file in files:
        mask = cv2.imread(os.path.join(mask_dir, file))
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

        unique = np.unique(mask.reshape(-1, 3), axis=0)
        for color in unique:
            colors.add(tuple(color))

    return sorted(list(colors))


unique_colors = get_unique_colors(TRAIN_MASK_DIR)
NUM_CLASSES = len(unique_colors)

print("Detected Classes:", NUM_CLASSES)


Detected Classes: 6


In [8]:
COLOR_MAP = {color: idx for idx, color in enumerate(unique_colors)}

def rgb_to_mask(mask):
    h, w, _ = mask.shape
    class_mask = np.zeros((h, w), dtype=np.uint8)

    for rgb, cls in COLOR_MAP.items():
        matches = np.all(mask == rgb, axis=-1)
        class_mask[matches] = cls

    return class_mask


In [9]:
class OffroadDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = sorted(os.listdir(img_dir))
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.mask_dir:
            mask_path = os.path.join(self.mask_dir, self.images[idx])
            mask = cv2.imread(mask_path)
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
            mask = rgb_to_mask(mask)
        else:
            mask = None

        if self.transform:
            if mask is not None:
                augmented = self.transform(image=image, mask=mask)
                image = augmented["image"]
                mask = augmented["mask"]
            else:
                augmented = self.transform(image=image)
                image = augmented["image"]

        if mask is not None:
            return image.float(), mask.long()
        else:
            return image.float(), self.images[idx]


In [10]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 4
EPOCHS = 20
LR = 3e-4
IMAGE_SIZE = 384


In [11]:
model = smp.DeepLabV3Plus(
    encoder_name="efficientnet-b2",
    encoder_weights="imagenet",
    classes=NUM_CLASSES,
    activation=None,
).to(DEVICE)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/106 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/36.8M [00:00<?, ?B/s]

In [12]:
train_transform = A.Compose([
    A.Resize(IMAGE_SIZE, IMAGE_SIZE),
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussNoise(p=0.2),
    A.Normalize(),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(IMAGE_SIZE, IMAGE_SIZE),
    A.Normalize(),
    ToTensorV2(),
])


In [13]:
train_dataset = OffroadDataset(TRAIN_IMG_DIR, TRAIN_MASK_DIR, train_transform)
val_dataset = OffroadDataset(VAL_IMG_DIR, VAL_MASK_DIR, val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [14]:
def compute_class_weights(loader, num_classes):
    pixel_counts = np.zeros(num_classes)

    for images, masks in tqdm(loader):
        masks = masks.numpy()
        for cls in range(num_classes):
            pixel_counts[cls] += np.sum(masks == cls)

    weights = 1.0 / (pixel_counts + 1e-6)
    weights = weights / weights.sum() * num_classes
    return torch.tensor(weights, dtype=torch.float32)

class_weights = compute_class_weights(train_loader, NUM_CLASSES).to(DEVICE)

ce_loss = torch.nn.CrossEntropyLoss(weight=class_weights)
dice_loss = smp.losses.DiceLoss(mode="multiclass")


100%|██████████| 714/714 [04:27<00:00,  2.67it/s]


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="max",
    factor=0.5,
    patience=3
)

scaler = torch.amp.GradScaler(enabled=(DEVICE=="cuda"))

In [16]:
def compute_per_class_iou(pred, mask):
    pred = torch.argmax(pred, dim=1)
    ious = []

    for cls in range(NUM_CLASSES):
        pred_inds = (pred == cls)
        target_inds = (mask == cls)

        intersection = (pred_inds & target_inds).sum().item()
        union = (pred_inds | target_inds).sum().item()

        if union == 0:
            ious.append(0)
        else:
            ious.append(intersection / union)

    return ious


In [17]:
best_iou = 0
patience = 5
counter = 0

for epoch in range(EPOCHS):

    model.train()
    train_loss = 0

    for images, masks in tqdm(train_loader):
        images, masks = images.to(DEVICE), masks.to(DEVICE)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            preds = model(images)
            loss = dice_loss(preds, masks) + ce_loss(preds, masks)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()

    model.eval()
    all_class_ious = np.zeros(NUM_CLASSES)

    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            preds = model(images)

            class_ious = compute_per_class_iou(preds, masks)
            all_class_ious += np.array(class_ious)

    all_class_ious /= len(val_loader)
    val_iou = np.mean(all_class_ious)

    scheduler.step(val_iou)

    print(f"\nEpoch {epoch+1}")
    print("Train Loss:", train_loss / len(train_loader))
    print("Per-class IoU:", np.round(all_class_ious, 4))
    print("Mean IoU:", val_iou)

    if val_iou > best_iou:
        best_iou = val_iou
        torch.save(model.state_dict(), "best_balanced_model.pth")
        print("Model Improved & Saved")
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

print("Best IoU:", best_iou)


  0%|          | 0/714 [00:00<?, ?it/s]/tmp/ipython-input-1049/3360852238.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
100%|██████████| 714/714 [05:09<00:00,  2.30it/s]



Epoch 1
Train Loss: 1.5022042153262292
Per-class IoU: [0.421  0.5092 0.2339 0.1129 0.4063 0.8995]
Mean IoU: 0.4304484599737921
Model Improved & Saved


100%|██████████| 714/714 [04:54<00:00,  2.42it/s]



Epoch 2
Train Loss: 1.234190928752349
Per-class IoU: [0.4497 0.5483 0.27   0.1546 0.4754 0.8997]
Mean IoU: 0.46627232656814477
Model Improved & Saved


100%|██████████| 714/714 [04:54<00:00,  2.43it/s]


...


Epoch 18
Train Loss: 0.901033947280809
Per-class IoU: [0.5926 0.6133 0.3723 0.2734 0.5515 0.9036]
Mean IoU: 0.5511043337929353


100%|██████████| 714/714 [04:55<00:00,  2.41it/s]



Epoch 19
Train Loss: 0.9049016151274619
Per-class IoU: [0.5944 0.6119 0.3691 0.2724 0.5486 0.9037]
Mean IoU: 0.5500274081480551


100%|██████████| 714/714 [04:57<00:00,  2.40it/s]



Epoch 20
Train Loss: 0.8958697651280743
Per-class IoU: [0.6004 0.6151 0.3728 0.2705 0.5545 0.9037]
Mean IoU: 0.5528210951948301
Model Improved & Saved
Best IoU: 0.5528210951948301


In [21]:
OUTPUT_DIR = "/content/predictions"
os.makedirs(OUTPUT_DIR, exist_ok=True)

model.eval()

test_dataset = OffroadDataset(TEST_IMG_DIR, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

with torch.no_grad():
    for images, names in test_loader:
        images = images.to(DEVICE)
        preds = model(images)

        pred_mask = torch.argmax(preds, dim=1).squeeze().cpu().numpy()

        rgb_mask = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3), dtype=np.uint8)

        for color, idx in COLOR_MAP.items():
            rgb_mask[pred_mask == idx] = color

        cv2.imwrite(os.path.join(OUTPUT_DIR, names[0]),
                    cv2.cvtColor(rgb_mask, cv2.COLOR_RGB2BGR))

print("Test predictions saved.")

Test predictions saved.


In [22]:
!zip -r predictions.zip /content/predictions


updating: content/predictions/ (stored 0%)
updating: content/predictions/0000582.png (deflated 25%)
updating: content/predictions/0000105.png (deflated 25%)
updating: content/predictions/0000977.png (deflated 25%)
updating: content/predictions/0000491.png (deflated 24%)
updating: content/predictions/0000459.png (deflated 26%)
updating: content/predictions/0000324.png (deflated 26%)
updating: content/predictions/0001044.png (deflated 24%)
updating: content/predictions/0000572.png (deflated 25%)
updating: content/predictions/0000222.png (deflated 25%)
updating: content/predictions/0000060.png (deflated 24%)
updating: content/predictions/0000609.png (deflated 24%)
updating: content/predictions/0000544.png (deflated 24%)
updating: content/predictions/0000090.png (deflated 26%)
updating: content/predictions/0000687.png (deflated 25%)
updating: content/predictions/0000608.png (deflated 25%)
updating: content/predictions/0000412.png (deflated 26%)
updating: content/predictions/0000627.png (de

In [23]:
from google.colab import files
files.download('predictions.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>